## ДЗ1
### Такташева Екатерина, БКЛ182

Импортируем модуль JSON:

In [103]:
import json

Считываем файл в список словарей:

In [104]:
twitter = []
for line in open('hw_3_twitter.json'):
    twitter.append(json.loads(line))

### Задание 1: Количество твитов (и ретвитов) в наборе

In [105]:
number_of_tweets = len(twitter)

#### Ответ: 

In [106]:
print('Колчество твитов равно', number_of_tweets)

Колчество твитов равно 2556


### Задание 2: Процент удаленных твитов
Создаем два пустых списка normal и deleted, перебираем словари-твиты, если в словаре встречается ключ 'delete', то добавляем его в соответсвующий список

In [107]:
normal = []
deleted = []
for i in range(len(twitter)):
    if 'delete' in twitter[i].keys():
        deleted.append(twitter[i])
    else:
        normal.append(twitter[i])

#### Ответ: 

In [108]:
print('Процент удаленных твитов равен', 
      len(deleted) / number_of_tweets * 100)

Процент удаленных твитов равен 14.162754303599373


### Задание 3: Самые популярные языки твитов
Импортируем модуль Counter

In [109]:
from collections import Counter

Создаем пустой список languages, перебираем неудалённые твиты, добавляем в список значение ключа 'lang', создаем словарь

In [110]:
languages = []
for i in range(len(normal)):
    languages.append(normal[i]['lang'])
dic = Counter(languages)

Сортируем словарь по частотности и распечатываем 10 самых популярных языков

In [111]:
common_lang = '№:\tЯзык:\tКол-во твитов:\n'  # заголовок
i = 1
for lang, count in dic.most_common(10):
    common_lang += (str(i) + '\t' + lang + 
                    '\t' + str(count) + '\n')
    i += 1

#### Ответ:

In [112]:
print('Список самых популярных языков:\n\n', 
      common_lang, sep='')

Список самых популярных языков:

№:	Язык:	Кол-во твитов:
1	en	719
2	ja	438
3	es	173
4	ko	149
5	th	123
6	ar	119
7	und	117
8	in	71
9	pt	69
10	fr	35



### Задание 4: Твиты одного и того же пользователя
Для начала создадим пустой спиок user_id:

In [113]:
user_id = []

Добавим в список все id пользователей из неудалённых твитов:

In [114]:
for i in range(len(normal)):
    user_id.append(normal[i]['user']['id_str'])

Теперь добавим в список все id пользователей из удалённых твитов:

In [115]:
for i in range(len(deleted)):
    user_id.append(deleted[i]['delete']['status']['user_id_str'])

Создадим словарь Counter, чтобы посчитать количество совпавших id, создадим переменную-счётчик user_match, которая будет считать количество совпадающих id:

In [116]:
users = Counter(user_id)
user_match = 0
for value in users.values():
    if value > 1:
        user_match += 1

#### Ответ:

In [117]:
print("""Количество пользователей, написавших более одного твита:""",
      user_match)

Количество пользователей, написавших более одного твита: 66


### Задание 5: Топ-20 хэштегов
Создаем пустой список, перебираем, находим ключ 'hashtag' и добавляем в список. 

In [118]:
hashtag_list = []
for i in range(len(normal)):
    hashtag = normal[i]['entities']['hashtags']
    if len(hashtag) != 0:
        for j in range(len(hashtag)):
            hashtag_list.append(hashtag[j]['text'])

Создаем переменную для вывода, добавляем в нее топ-20 хэштегов

In [119]:
common_hashtags = '№:\tХэштег:\tКол-во:\n'
i = 1
for tag, count in Counter(hashtag_list).most_common(20):
    common_hashtags += (str(i) + '\t' + tag + '\t' 
                        + str(count) +'\n')
    i += 1

#### Ответ:

In [120]:
print('Список самых популярных хэштегов:\n\n', 
      common_hashtags, sep='')

Список самых популярных хэштегов:

№:	Хэштег:	Кол-во:
1	BTS	17
2	방탄소년단	13
3	AMAs	11
4	人気投票ガチャ	8
5	태형	7
6	뷔	6
7	BTSinChicago	5
8	BTSLoveYourselfTour	5
9	오늘의방탄	5
10	PledgeForSwachhBharat	5
11	MPN	5
12	PCAs	4
13	V	4
14	시카고1회차공연	4
15	เป๊กผลิตโชค	4
16	JIMIN	4
17	running	3
18	NCT	3
19	지민	3
20	WajahmuPlastik	3



### Задание 6: Частотный словарь твитов на английском языке
##### При поддержке Ани Аксёновой
NB: я не стала использовать строку string.punctuation, т.к. она включает апострофы (которые все же являются частью слова), но игнорирует некоторые знаки пунктуации (например, многоточие)

Импортируем модуль re:

In [121]:
import re

Создадим списки, в которые положим все знаки, которые хотим удалить:

NB: Еще я заметила, что не все ставят одинаковые апострофы (' или ’), поэтому решила заморочиться и привести из все к одному виду, то же самое с тире и дефисами

In [282]:
punct = ['‐ ', ' ‐', ' ‐ ', "’ ", " ’"]
word_pattern = r"[^-A-Z’a-z\s]"
link_pattern = r'http(s)?.*?(\s|$)'  
handle_pattern = r'(@.*?(\s|$))'
hashtag_pattern = r'(#.+?(\s|$))'

Создаем список твитов на анлийском языке (не включающий ретвиты):

In [283]:
original_tweets = []
for i in range(len(normal)):
    if ('retweeted_status' not in normal[i].keys() 
        and normal[i]['lang'] == 'en'):
        original_tweets.append(normal[i]['text'])

Для начала удалим все эмоджи, ссылки, хэндлы пользователей и пунктуацию:

In [293]:
parced_tweets = []
for tweet in original_tweets:
    tweet = re.sub(r"'", '’', tweet)  # замена апострофа
    tweet = re.sub(r'—|-', '‐', tweet)  # замена дефисов и тире
    tweet = re.sub(handle_pattern, ' ', tweet)  # удаляем хэндлы 
    tweet = re.sub(link_pattern, ' ', tweet)  # удаляем ссылки
    tweet = re.sub(hashtag_pattern, ' ', tweet)  # удаляем хэштеги
    for char in punct:
        tweet = tweet.strip()
        tweet = tweet.replace(char, '')  # удаляем странные знаки
        tweet = re.sub(word_pattern, ' ', tweet)  # удаляем не слова
    tweet = tweet.lower().split()
    parced_tweets.extend(tweet)

Создадим частотный словарь и "положим" первые 20 самых частотных элементов в переменную: 

In [294]:
parced_counter = '№:\tСлово:\tКол-во вхождений:\n'
i = 1
for word, count in Counter(parced_tweets).most_common(20):  
    parced_counter += (str(i) + '\t' + word + 
                       '\t' + str(count) + '\n')
    i += 1

#### Ответ:

In [295]:
print('Самые частотные слова:\n\n', parced_counter, sep='')

Самые частотные слова:

№:	Слово:	Кол-во вхождений:
1	the	126
2	to	86
3	a	83
4	i	75
5	and	65
6	is	51
7	you	48
8	of	46
9	for	42
10	it	42
11	in	38
12	that	33
13	this	32
14	my	30
15	me	28
16	be	26
17	on	26
18	what	21
19	are	21
20	so	20



### Задание 7: количество подписчиков

Для начала "достанем" данные о подписчиках в список followers, при том будем записывать в него кортежи вида (username, number of followers):

In [287]:
followers = []
for i in range(len(normal)):
    followers.append((normal[i]['user']['name'], 
                      normal[i]['user']['followers_count']))

Создадим пустой словарь-счетчик и, пройдясь по кортежам из списка, добавим в него данные о подписчиках:

In [279]:
followers_dict = Counter()
for pare in followers:
    followers_dict[pare[0]] = pare[1]

Запишем данные о 10 самых популярных пользователях в переменную followers_counter:

In [280]:
followers_counter = '№:\tИмя пользователя:\tКол-во подписчиков:\n'
i = 1
for name, count in followers_dict.most_common(10):
    followers_counter += (str(i) + '\t' + name 
                          + '\t' + str(count) + '\n')
    i += 1

#### Ответ:

In [281]:
print('Самые популярные пользователи:\n\n',
      followers_counter, sep='')

Самые популярные пользователи:

№:	Имя пользователя:	Кол-во подписчиков:
1	Filosofía♕	2521403
2	FITNESS Magazine	1491309
3	malaysiakini.com	1206759
4	NYT Science	1137374
5	Gramática	625463
6	TGRT Haber	392472
7	The Sun Football ⚽	383698
8	Melbourne, Australia	374222
9	Roznama Express	318189
10	💞 ცųཞɠɛཞცơơɠıɛ 💞	311319



### Задание 8: Самые популярные источники

Запишем все источники в отдельный список list_of_sources:

In [270]:
list_of_sources = []
for i in range(len(normal)):
    list_of_sources.append(normal[i]['source'])               

При помощи регулярного выражения "вытащим" из тэга source собственно сам источник твита:

In [248]:
sources = []
source_pattern = r'(?:>(.+?)</a>)'
for tag in list_of_sources:
    sources.append(re.search(source_pattern, tag)[1])

Создадим словарь-счетчик и запишем 10 самых популярных источников в переменную sources_counter:

In [249]:
sources_counter = '№:\tИсточник:\tКол-во записей:\n'
i = 1
for source, count in Counter(sources).most_common(10):
    sources_counter += (str(i) + '\t' + source 
                        + '\t' + str(count) + '\n')
    i += 1

#### Ответ:

In [250]:
print('Самые популярные источники:\n\n',
      sources_counter, sep='')

Самые популярные источники:

№:	Источник:	Кол-во записей:
1	Twitter for iPhone	800
2	Twitter for Android	695
3	Twitter Web Client	140
4	twittbot.net	122
5	Twitter Lite	51
6	Twitter for iPad	28
7	TweetDeck	23
8	Facebook	17
9	IFTTT	14
10	تطبيق قرآني	10

